In [ ]:
from google.colab import files
import pandas as pd

In [ ]:
_ = files.upload()

Saving similarity.json to similarity.json


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# **STEP 1: FIND ECONOMIC TWEETS**

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.4 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

In [ ]:
def is_economy(text):
  # Load the model
  model = pipeline('zero-shot-classification', model='valhalla/distilbart-mnli-12-1',device=0 if torch.cuda.is_available() else -1)
  #print(device)
  candidate_labels = ['economy', 'not economy']

  # Use the model to perform zero-shot classification
  result = model(text, candidate_labels)

  result_dict={}
  # Print the result
  for label, score in zip(result['labels'], result['scores']):
    result_dict[label]=score
  
  if result_dict['economy']>result_dict['not economy']:
    return True
  else: return False

In [ ]:
print(is_economy("BREAKING: The House has passed the economic stimulus bill. Update to come."))
print(is_economy("Agreement Boosts Citi Oversight http://tinyurl.com/cbslry"))

True
False


# **STEP 2: FIND TWEETS THAT ARE PARAPHRASE OF (PART OF) A STATEMENT**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Prompsit/paraphrase-bert-en")
model = AutoModelForSequenceClassification.from_pretrained("Prompsit/paraphrase-bert-en").to(device)

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def split_text(text):
  doc = nlp(text)
  return list(doc.sents)

In [ ]:
def is_paraphrase(statement,tweet):
  doc1=split_text(statement)
  doc2=split_text(tweet)

  soft = torch.nn.Softmax(dim=1)

  count=0
  for sentence_target in doc1:
    for sentence_tweet in doc2:
      input = tokenizer(str(sentence_target),str(sentence_tweet),return_tensors='pt').to(device)
      logits = model(**input).logits
      x=soft(logits).tolist()[0]
      prob_0=x[0]
      prob_1=x[1]
      if prob_1>=0.9:
        count=count+1
  if count >=1:
    return True
  else:
    return False

# **STEP 3: CHECK IF THE TWEETS MENTION DIRECTLY THE FED**

In [ ]:
def speaks_fed(text):
  synonyms = []

  for syn in wordnet.synsets("Federal_Reserve_System"):
      for l in syn.lemmas():
          synonyms.append(l.name())
  
  res=[]
  for term in synonyms:
    res.append(term.split('_'))

  fed_words=[item.lower() for sublist in res for item in sublist]
  fed_words=set(fed_words)

  flag=False

  for w in nltk.word_tokenize(text):
      if w.lower() in list(fed_words):
        flag=True
        break
  return flag

# **STEP 4: PUT ALL TOGETHER FOR LABELING**

In [ ]:
def decide_label(similarity,is_paraphrase,speak_fed,is_economy):
  if (similarity>=0.4 and speak_fed==True) or (is_paraphrase==True):
    return 1
  elif (similarity<0.4 and speak_fed==True) or (is_economy==True) :
    return 2
  elif (is_economy==False):
    return 3

In [ ]:
import json 

#f = open('similarity.json')
#similarity = json.load(f)
#f.close()

# Load the JSON file into a dictionary
with open('similarity.json', 'r') as f:
    similarity = json.load(f)

In [ ]:
def decide_label2(similarity,is_paraphrase,speak_fed,is_economy):
  if (speak_fed==True) or ((is_paraphrase==1) and (is_economy==True)):
    return 1
  elif (similarity>=0.4) or ((is_economy==True)):
    return 2
  else:
    return 3

In [ ]:
import nltk
from nltk.corpus import wordnet
from tqdm import tqdm
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

for key, value in similarity.items() :
  if key>="1":
    break
  for i in tqdm(range(0,len(value['related tweets']))):
    if value['related tweets'][i][3]<0.2:
      label=3
    else:
      label=decide_label2(value['related tweets'][i][3],
                                  is_paraphrase(value['statement'],value['related tweets'][i][2]),
                                  speaks_fed(value['related tweets'][i][2]),
                                  is_economy(value['related tweets'][i][2])
                                  )
    value['related tweets'][i].append(label)
    with open('similarity.json', 'w') as f:
      json.dump(similarity, f, indent=4)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
100%|██████████| 117/117 [05:04<00:00,  2.61s/it]
